In [328]:
import pandas as pd
import numpy as np
import requests
import json
import re
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')


## For save/read files

In [243]:
def read_file(file_name):
    df = pd.read_csv(file_name, sep=';')
    pd.set_option('display.max_columns', 125)
    return df

In [244]:
def save_file(all_avto_links, file_name):
    df = pd.DataFrame({'pages_href': all_avto_links}, index=range(1, len(all_avto_links)+1))
    df = df.rename_axis('id_car')
    df.head(100)

    df.to_csv(file_name, sep=';')

## For parse links of announcement of cars

In [245]:
def parse_links(url, all_avto_links):
    response = requests.get(url)
    
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    avtos = soup.find_all('div', {'class':'listing-item'})
    
    for avto in avtos:
        class_pages_href = avto.find('a')['href']    
        all_avto_links.append(class_pages_href)
        
    return all_avto_links
    

In [232]:
url = 'https://cars.av.by/search/page/'

file_name = 'data/auto12.csv'

all_avto_links = []
    
for page in range(1100, 1200):
    all_avto_links = parse_links(url+str(page), all_avto_links)
    

save_file(all_avto_links, file_name)
read_file(file_name)

      id_car                                       pages_href
0          1              https://cars.av.by/audi/a6/16520163
1          2    https://cars.av.by/volkswagen/passat/16751766
2          3  https://cars.av.by/jeep/grand-cherokee/16292976
3          4           https://cars.av.by/fiat/punto/16710227
4          5      https://cars.av.by/chevrolet/cruze/16063909
...      ...                                              ...
2495    2496            https://cars.av.by/mazda/626/16748595
2496    2497      https://cars.av.by/audi/a6-allroad/16025687
2497    2498      https://cars.av.by/ford/fusion-usa/16710455
2498    2499       https://cars.av.by/toyota/corolla/16750629
2499    2500            https://cars.av.by/volvo/v70/16750628

[2500 rows x 2 columns]


## For parse data 

In [266]:
def parse(filename):    
    return ''
        
    
filename = 'data/auto1.csv'

df_link = read_file(filename)
df_link['pages_href'] = df_link['pages_href'].apply(str)
url = df_link['pages_href'][1]
multi_class = 'card'

In [444]:
def parse(url):
    response = requests.get(url)
    response.encoding = 'utf8'

    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    avtos = soup.find_all('div', {'class':'card'})


    avto = []
    for elem in avtos:
        name = re.sub(r'<h1 class=\"card-title\">|</h1>|\n*', 
                      r'', str(elem.find('h1', {'class':'card-title'})).strip())

        price = re.sub(r'[<>a-zA-Z\/\"\'\=\-\+\_\@\#\/\.\n\ ]*', 
                       r'',str(elem.find('span', {'class':'card-price-main-primary'})).strip())

        main_info = str(elem.find('div', {'class':'card-info'})).strip()
        options = re.sub(r'[<>a-zA-Z\/\"\'\=\-\+\_\@\#\/\.\,]', r'', main_info)
        fin_str = re.sub(r'[\n]', r' ', options).strip()
        options = fin_str.split('     ')

        town = re.sub(r'<h4 class=\"card-contacts-city\">|</h4>|\n*', r'', 
                      str(elem.find('h4', {'class':'card-contacts-city'})).strip())

        add_f = re.sub(r'<div class=\"card-options js-card-options\">|<h4>|<\/h4>|<h5>|<\/h5>|<div>|<\/div>|<ul>|<\/ul>|<\/li>|Дополнительные опции:|Безопасность|Климат и обогрев|Комфорт|Мультимедиа|Экстерьер|Оптика и свет|\n*', r'', 
                       str(elem.find('div', {'class':'card-options js-card-options'})).strip())
        add_f = add_f.split('<li>')

        avto.append([name, price, options, town, add_f])

        df = pd.DataFrame(avto)
    
        df.head(10)
        
parse(url)